In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 41.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=c69e5fcb3206577cebf43bcb428858a4fafad68ffde066bd9f162f993457e8a9
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1f_9EbnywCj35EBUA32sueigxBjBJwALr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('War and Peace by Leo Tolstoy (ru).txt')

In [4]:
import pandas as pd

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [5]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
spark

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-12-22 09:40:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.204.93.39, 34.195.187.253, 52.202.213.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.204.93.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  37.9MB/s    in 0.3s    

2020-12-22 09:41:00 (37.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


Подсчитай кол-во слов в документе "War and Peace by Leo Tolstoy (ru).txt"

In [8]:
# код
# перекодировка в utf_8
f = open('War and Peace by Leo Tolstoy (ru).txt', 'r',encoding='cp1251')
text = f.read().encode("utf-8").decode("utf-8")

utf_8_text = open("utf_8_text.txt", "w")
utf_8_text.write(text)
utf_8_text.close()

In [9]:
def lower_clean(x):
  punc='!"#$%\'&*(),.;<=>?@[\\]^_`{|}~-'
  lowered = x.lower()
  for ch in punc:
    lowered = lowered.replace(ch,'')
  lowered = lowered.replace('\t','')
  return lowered

In [10]:
book = sc.textFile('utf_8_text.txt').map(lower_clean) # читаем файл и чистим
clean_book = book.flatMap(lambda line: line.split(" ")) # разделяем на слова --> лист слов 
clean_book = clean_book.filter(lambda word: word != '') # удаляем пустые
text = clean_book.map(lambda word: (word,1)) # мэпим для каждого слова единичку --> лист "слово - единица"
reduced_text = text.reduceByKey(lambda x,y: x+y) # для всех одинаковых ключей(слов) складываем значения --> количество повторений каждого слова
Sum = reduced_text.map(lambda word: word[1]).sum() # 

In [11]:
print(reduced_text.take(3))
print(Sum)

[('война', 71), ('и', 21470), ('мир', 50)]
474044


In [15]:
id='13yfAoONwq4rS5XrTv3IrcqcFcdgfvK9V'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1VE_9x0LQvOJpHXbXp_RMPl3Q4wRUuOok'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

Необходимо обучить модель используя Spark MLlib (модель на ваш выбор, например Decision Tree) и получить accuracy.
Подробнее тут: https://spark.apache.org/docs/latest/ml-classification-regression.html

In [16]:
# код

train = spark.read.format("libsvm").option("numFeatures","780").load('mnist-digits-train.txt')
test = spark.read.format("libsvm").option("numFeatures","780").load('mnist-digits-test.txt')

In [17]:
len(test.head()[1])

780

In [8]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

In [ ]:
#******* not needed
from pyspark.sql.functions import UserDefinedFunction
import pyspark.ml.linalg
def add_2_empty_col_(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    new_array.append(0)
    new_array.append(0)
    v = pyspark.ml.linalg.SparseVector(new_array)
    return v
name = 'features'
udf = UserDefinedFunction(lambda x: add_2_empty_col_(x),T.ArrayType(T.FloatType()))
test_test = test.select(*[udf(column).alias(name) if column == name else column for column in test.columns])

In [20]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
labelIndexer = StringIndexer(inputCol='label',outputCol='indexedLabel').fit(train)
featureIndexer = VectorIndexer(inputCol='features',outputCol='indexedFeatures',handleInvalid='skip').fit(train)
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(train)

# Make predictions.
predictions = model.transform(test)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

treeModel = model.stages[2]
# summary only
print(treeModel)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(780,[202,203,204...|
|       6.0|         3.0|(780,[94,95,96,97...|
|       0.0|         0.0|(780,[128,129,130...|
|       5.0|         5.0|(780,[124,125,126...|
|       4.0|         8.0|(780,[150,151,159...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0.321601 
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5ae3b83eefd6, depth=5, numNodes=59, numClasses=10, numFeatures=780


In [9]:
id='1kUIrskM0zNH8u71G9M1BkHjRQYxvgAvh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data.zip')
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/regions.csv        
  inflating: data/departments.csv    
  inflating: data/jobs.csv           
  inflating: data/locations.csv      
  inflating: data/country.csv        
  inflating: data/employees.csv      
  inflating: data/job_history.csv    


Кто получает больше всего? Кто меньше всего?

In [ ]:
pd.read_csv("data/jobs.csv") 

In [11]:
jobs_df = spark.read.option("delimiter", "\t").load("data/jobs.csv", format="csv")
jobs_df = jobs_df.withColumnRenamed("_c0", "JOB_ID")
jobs_df = jobs_df.withColumnRenamed("_c1", "JOB_TITLE")
jobs_df = jobs_df.withColumnRenamed("_c2", "MIN_SALARY")
jobs_df = jobs_df.withColumnRenamed("_c3", "MAX_SALARY")

In [12]:
jobs_df.createOrReplaceTempView("JOBS_DF")
sqlDF = spark.sql("SELECT JOB_TITLE, ((MAX_SALARY+MIN_SALARY)/2) AS MEAN_SALARY FROM JOBS_DF ORDER BY MEAN_SALARY DESC")

In [36]:
sqlDF.collect()

[Row(JOB_TITLE='President', MEAN_SALARY=30040.0),
 Row(JOB_TITLE='Administration Vice President', MEAN_SALARY=22500.0),
 Row(JOB_TITLE='Sales Manager', MEAN_SALARY=15040.0),
 Row(JOB_TITLE='Finance Manager', MEAN_SALARY=12100.0),
 Row(JOB_TITLE='Accounting Manager', MEAN_SALARY=12100.0),
 Row(JOB_TITLE='Marketing Manager', MEAN_SALARY=12000.0),
 Row(JOB_TITLE='Purchasing Manager', MEAN_SALARY=11500.0),
 Row(JOB_TITLE='Sales Representative', MEAN_SALARY=9004.0),
 Row(JOB_TITLE='Public Relations Representative', MEAN_SALARY=7500.0),
 Row(JOB_TITLE='Stock Manager', MEAN_SALARY=7000.0),
 Row(JOB_TITLE='Programmer', MEAN_SALARY=7000.0),
 Row(JOB_TITLE='Accountant', MEAN_SALARY=6600.0),
 Row(JOB_TITLE='Public Accountant', MEAN_SALARY=6600.0),
 Row(JOB_TITLE='Marketing Representative', MEAN_SALARY=6500.0),
 Row(JOB_TITLE='Human Resources Representative', MEAN_SALARY=6500.0),
 Row(JOB_TITLE='Administration Assistant', MEAN_SALARY=4500.0),
 Row(JOB_TITLE='Purchasing Clerk', MEAN_SALARY=4000.0),

Выведете топ 5 по зарплате.

In [37]:
sqlDF = spark.sql("SELECT JOB_TITLE, ((MAX_SALARY+MIN_SALARY)/2) AS MEAN_SALARY FROM JOBS_DF ORDER BY MEAN_SALARY DESC LIMIT 5")
sqlDF.collect()

[Row(JOB_TITLE='President', MEAN_SALARY=30040.0),
 Row(JOB_TITLE='Administration Vice President', MEAN_SALARY=22500.0),
 Row(JOB_TITLE='Sales Manager', MEAN_SALARY=15040.0),
 Row(JOB_TITLE='Finance Manager', MEAN_SALARY=12100.0),
 Row(JOB_TITLE='Accounting Manager', MEAN_SALARY=12100.0)]

Подгружаем оставшиеся таблицы

In [ ]:
pd.read_csv("data/regions.csv", sep='\t')

In [ ]:
pd.read_csv("data/country.csv", sep='\t').head()

In [ ]:
pd.read_csv("data/locations.csv", sep='\t').head()

In [ ]:
pd.read_csv("data/departments.csv", sep='\t').head()

In [64]:
pd.read_csv("data/employees.csv", sep='\t', comment='#').head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17.06.03,AD_PRES,24000,NaN,NaN,90
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21.09.05,AD_VP,17000,NaN,100.0,90
2,102,Lex,De Haan,LDEHAAN,515.123.4569,13.01.01,AD_VP,17000,NaN,100.0,90
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,03.01.06,IT_PROG,9000,NaN,102.0,60
4,104,Bruce,Ernst,BERNST,590.423.4568,21.05.07,IT_PROG,6000,NaN,103.0,60


In [15]:
regions_df = spark.read.option("delimiter", "\t").load("data/regions.csv", format="csv").toDF(
  "REGION_ID", "REGION_NAME"
)
regions_df.createOrReplaceTempView("regions_df")
country_df = spark.read.option("delimiter", "\t").load("data/country.csv", format="csv").toDF(
  "COUNTRY_ID", "COUNTRY_NAME", "REGION_ID"
)
country_df.createOrReplaceTempView("country_df")
locations_df = spark.read.option("delimiter", "\t").load("data/locations.csv", format="csv").toDF(
  "LOCATION_ID", "STREET_ADDRESS", "POSTAL_CODE", "CITY", "STATE_PROVINCE", "COUNTRY_ID"
)
locations_df.createOrReplaceTempView("locations_df")
departments_df = spark.read.option("delimiter", "\t").load("data/departments.csv", format="csv").toDF(
  "DEPARTMENT_ID", "DEPARTMENT_NAME", "MANAGER_ID", "LOCATION_ID"
)
departments_df.createOrReplaceTempView("departments_df")
employees_df = spark.read.option("delimiter", "\t").load("data/employees.csv", format="csv").toDF(
  "EMPLOYEE_ID", "FIRST_NAME", "LAST_NAME", "EMAIL", "PHONE_NUMBER", "HIRE_DATE", "JOB_ID", "SALARY", "COMMISSION_PCT", "MANAGER_ID", "DEPARTMENT_ID"
)
employees_df.createOrReplaceTempView("employees_df")

Сколько всего регионов? Сколько работников в каждом регионе?

Связь regions - country - location - department - employer


In [71]:
sqlDF = spark.sql("\
  SELECT DISTINCT REGION_ID\
  FROM regions_df")
sqlDF.collect()

[Row(REGION_ID='3'),
 Row(REGION_ID='REGION_ID'),
 Row(REGION_ID='1'),
 Row(REGION_ID='4'),
 Row(REGION_ID='2')]

In [70]:
sqlDF = spark.sql("\
  SELECT  r.REGION_ID,COUNT(EMPLOYEE_ID) \
  FROM employees_df as e \
  JOIN departments_df as d on e.DEPARTMENT_ID = d.DEPARTMENT_ID\
  JOIN locations_df as l on d.LOCATION_ID = l.LOCATION_ID\
  JOIN country_df as c on l.COUNTRY_ID = c.COUNTRY_ID\
  JOIN regions_df as r on c.REGION_ID = r.REGION_ID\
  GROUP BY r.REGION_ID")
sqlDF.collect()

[Row(REGION_ID='REGION_ID', count(EMPLOYEE_ID)=1),
 Row(REGION_ID='1', count(EMPLOYEE_ID)=36),
 Row(REGION_ID='2', count(EMPLOYEE_ID)=70)]

Выведете всех работников из Китая.

In [16]:
sqlDF = spark.sql("\
  SELECT  e.EMPLOYEE_ID,c.COUNTRY_NAME  \
  FROM employees_df as e \
  JOIN departments_df as d on e.DEPARTMENT_ID = d.DEPARTMENT_ID\
  JOIN locations_df as l on d.LOCATION_ID = l.LOCATION_ID\
  JOIN country_df as c on l.COUNTRY_ID = c.COUNTRY_ID\
  WHERE c.COUNTRY_NAME = 'China'")
sqlDF.collect()

[]

Укажите самую высокооплачиваемою должность.

In [76]:
sqlDF = spark.sql("SELECT JOB_TITLE, ((MAX_SALARY+MIN_SALARY)/2) AS MEAN_SALARY FROM JOBS_DF ORDER BY MEAN_SALARY DESC LIMIT 1")
sqlDF.collect()

[Row(JOB_TITLE='President', MEAN_SALARY=30040.0)]

Выведете всех работников связанных с ИТ. Выведете их менеджеров. 

In [79]:
sqlDF = spark.sql("\
  SELECT  EMPLOYEE_ID,MANAGER_ID   \
  FROM employees_df as e \
  WHERE JOB_ID = 'IT_PROG'")
sqlDF.collect()

[Row(EMPLOYEE_ID='103', MANAGER_ID='102'),
 Row(EMPLOYEE_ID='104', MANAGER_ID='103'),
 Row(EMPLOYEE_ID='105', MANAGER_ID='103'),
 Row(EMPLOYEE_ID='106', MANAGER_ID='103'),
 Row(EMPLOYEE_ID='107', MANAGER_ID='103')]

Выведете имя и фамилию работника, его текущую и предыдущую должности и сколько полных недель и дней прошло с момент изменения. 



In [161]:
job_history_df = spark.read.option("delimiter", "\t").load("data/job_history.csv", format="csv").toDF(
  "EMPLOYEE_ID", "START_DATE", "END_DATE", "JOB_ID", "DEPARTMENT_ID"
)
job_history_df=job_history_df.where("START_DATE!='START_DATE'")# дропаем лишнюю строку


In [ ]:
pd.read_csv("data/job_history.csv", sep='\t', comment='#')

In [162]:
# Необходимо исправить дату для того, чтобы пользоваться функциями sql
from datetime import datetime
import pyspark.sql.types as T
import pyspark.sql.functions as F

def user_defined_timestamp(date_col):
    _date = str(date_col).strip().split('.')
    l = len(_date)
    temp = _date[0]
    if(int(_date[l-1])<50):
      _date[0] = '20'+_date[l-1]
    else:
      _date[0] = '19'+_date[l-1]
    _date[l-1]=temp
    return _date[0]+'-'+_date[l-2]+'-'+_date[l-1]

user_defined_timestamp_udf = F.udf(user_defined_timestamp, T.StringType())

In [163]:
from pyspark.sql.functions import date_format
#df.select(date_format('dt', 'ddMMyy').alias('date')).collect()
job_history_df = job_history_df.withColumn('NEW_START_DATE', user_defined_timestamp_udf('START_DATE'))
job_history_df = job_history_df.withColumn('NEW_END_DATE', user_defined_timestamp_udf('END_DATE'))
job_history_df.createOrReplaceTempView("job_history_df")

In [104]:
job_history_df.collect()

[Row(EMPLOYEE_ID='102', START_DATE='13.01.01', END_DATE='24.07.06', JOB_ID='IT_PROG', DEPARTMENT_ID='60', NEW_START_DATE='2001.01.13', NEW_END_DATE='2006.07.24'),
 Row(EMPLOYEE_ID='101', START_DATE='21.09.97', END_DATE='27.10.01', JOB_ID='AC_ACCOUNT', DEPARTMENT_ID='110', NEW_START_DATE='1997.09.21', NEW_END_DATE='2001.10.27'),
 Row(EMPLOYEE_ID='101', START_DATE='28.10.01', END_DATE='15.03.05', JOB_ID='AC_MGR', DEPARTMENT_ID='110', NEW_START_DATE='2001.10.28', NEW_END_DATE='2005.03.15'),
 Row(EMPLOYEE_ID='201', START_DATE='17.02.04', END_DATE='19.12.07', JOB_ID='MK_REP', DEPARTMENT_ID='20', NEW_START_DATE='2004.02.17', NEW_END_DATE='2007.12.19'),
 Row(EMPLOYEE_ID='114', START_DATE='24.03.06', END_DATE='31.12.07', JOB_ID='ST_CLERK', DEPARTMENT_ID='50', NEW_START_DATE='2006.03.24', NEW_END_DATE='2007.12.31'),
 Row(EMPLOYEE_ID='122', START_DATE='01.01.07', END_DATE='31.12.07', JOB_ID='ST_CLERK', DEPARTMENT_ID='50', NEW_START_DATE='2007.01.01', NEW_END_DATE='2007.12.31'),
 Row(EMPLOYEE_ID=

In [164]:
sqlDF = spark.sql("\
  SELECT EMPLOYEE_ID, CAST(NEW_START_DATE AS date) \
  FROM job_history_df \
  WHERE EMPLOYEE_ID = 101")
sqlDF.collect()

[Row(EMPLOYEE_ID='101', NEW_START_DATE=datetime.date(1997, 9, 21)),
 Row(EMPLOYEE_ID='101', NEW_START_DATE=datetime.date(2001, 10, 28))]

In [165]:
sqlDF = spark.sql("\
  SELECT EMPLOYEE_ID, MAX(NEW_START_DATE) AS START_OF_CUR \
  FROM job_history_df \
  GROUP BY EMPLOYEE_ID \
  ")
sqlDF.createOrReplaceTempView("CUR")
sqlDF.collect()

[Row(EMPLOYEE_ID='200', START_OF_CUR='2002-07-01'),
 Row(EMPLOYEE_ID='101', START_OF_CUR='2001-10-28'),
 Row(EMPLOYEE_ID='201', START_OF_CUR='2004-02-17'),
 Row(EMPLOYEE_ID='102', START_OF_CUR='2001-01-13'),
 Row(EMPLOYEE_ID='176', START_OF_CUR='2007-01-01'),
 Row(EMPLOYEE_ID='122', START_OF_CUR='2007-01-01'),
 Row(EMPLOYEE_ID='114', START_OF_CUR='2006-03-24')]

In [166]:
sqlDF = spark.sql("\
  SELECT J.EMPLOYEE_ID, START_OF_CUR, J.JOB_ID  \
  FROM job_history_df AS J\
  JOIN  CUR AS C ON C.START_OF_CUR = J.NEW_START_DATE AND C.EMPLOYEE_ID = J.EMPLOYEE_ID\
  ")
sqlDF.createOrReplaceTempView("CUR")
sqlDF.collect()

[Row(EMPLOYEE_ID='102', START_OF_CUR='2001-01-13', JOB_ID='IT_PROG'),
 Row(EMPLOYEE_ID='101', START_OF_CUR='2001-10-28', JOB_ID='AC_MGR'),
 Row(EMPLOYEE_ID='201', START_OF_CUR='2004-02-17', JOB_ID='MK_REP'),
 Row(EMPLOYEE_ID='114', START_OF_CUR='2006-03-24', JOB_ID='ST_CLERK'),
 Row(EMPLOYEE_ID='122', START_OF_CUR='2007-01-01', JOB_ID='ST_CLERK'),
 Row(EMPLOYEE_ID='176', START_OF_CUR='2007-01-01', JOB_ID='SA_MAN'),
 Row(EMPLOYEE_ID='200', START_OF_CUR='2002-07-01', JOB_ID='AC_ACCOUNT')]

In [167]:
#Работники с несколькими должностями И ИХ ТЕК РАБОТА
sqlDF = spark.sql("\
  SELECT CUR.EMPLOYEE_ID,CUR.START_OF_CUR FROM (SELECT EMPLOYEE_ID, count(NEW_START_DATE) AS CNT \
  FROM job_history_df \
  GROUP BY EMPLOYEE_ID) \
  JOIN CUR ON CUR.EMPLOYEE_ID = __auto_generated_subquery_name.EMPLOYEE_ID\
  where CNT>1\
  ")
sqlDF.createOrReplaceTempView("TEMP")
sqlDF.collect()

[Row(EMPLOYEE_ID='200', START_OF_CUR='2002-07-01'),
 Row(EMPLOYEE_ID='101', START_OF_CUR='2001-10-28'),
 Row(EMPLOYEE_ID='176', START_OF_CUR='2007-01-01')]

In [168]:
# УДАЛИЛИ ТЕК РАБОТУ ТЕХ, У КОГО БЫЛО МНОГО ДОЛЖНОСТЕЙ
sqlDF = spark.sql("\
  SELECT EMPLOYEE_ID,NEW_START_DATE,NEW_END_DATE,JOB_ID \
  FROM job_history_df \
  WHERE (EMPLOYEE_ID,NEW_START_DATE) NOT IN (SELECT EMPLOYEE_ID,START_OF_CUR FROM TEMP)\
  ")
sqlDF.createOrReplaceTempView("PREV")
sqlDF.collect()

[Row(EMPLOYEE_ID='102', NEW_START_DATE='2001-01-13', NEW_END_DATE='2006-07-24', JOB_ID='IT_PROG'),
 Row(EMPLOYEE_ID='101', NEW_START_DATE='1997-09-21', NEW_END_DATE='2001-10-27', JOB_ID='AC_ACCOUNT'),
 Row(EMPLOYEE_ID='201', NEW_START_DATE='2004-02-17', NEW_END_DATE='2007-12-19', JOB_ID='MK_REP'),
 Row(EMPLOYEE_ID='114', NEW_START_DATE='2006-03-24', NEW_END_DATE='2007-12-31', JOB_ID='ST_CLERK'),
 Row(EMPLOYEE_ID='122', NEW_START_DATE='2007-01-01', NEW_END_DATE='2007-12-31', JOB_ID='ST_CLERK'),
 Row(EMPLOYEE_ID='200', NEW_START_DATE='1995-09-17', NEW_END_DATE='2001-06-17', JOB_ID='AD_ASST'),
 Row(EMPLOYEE_ID='176', NEW_START_DATE='2006-03-24', NEW_END_DATE='2006-12-31', JOB_ID='SA_REP')]

In [173]:
sqlDF = spark.sql("\
  SELECT * FROM (SELECT EMPLOYEE_ID,MAX(NEW_END_DATE) AS END_OF_PREV_DATE  \
  FROM PREV \
  GROUP BY EMPLOYEE_ID) \
  WHERE EMPLOYEE_ID IN (SELECT EMPLOYEE_ID FROM TEMP)\
  ")
sqlDF.createOrReplaceTempView("PREV_JOB")
sqlDF.collect()

[Row(EMPLOYEE_ID='200', END_OF_PREV_DATE='2001-06-17'),
 Row(EMPLOYEE_ID='101', END_OF_PREV_DATE='2001-10-27'),
 Row(EMPLOYEE_ID='176', END_OF_PREV_DATE='2006-12-31')]

In [174]:
# ПОЛУЧИЛИ ПРЕДЫДУЩИЕ ДОЛЖНОСТИ И ДАТЫ УВОЛЬНЕНИЯ
sqlDF = spark.sql("\
  SELECT P.EMPLOYEE_ID, END_OF_PREV_DATE, J.JOB_ID  \
  FROM PREV_JOB AS P\
  JOIN  job_history_df AS J ON P.END_OF_PREV_DATE = J.NEW_END_DATE AND P.EMPLOYEE_ID = J.EMPLOYEE_ID\
  ")
sqlDF.createOrReplaceTempView("PREV_JOB")
sqlDF.collect()

[Row(EMPLOYEE_ID='101', END_OF_PREV_DATE='2001-10-27', JOB_ID='AC_ACCOUNT'),
 Row(EMPLOYEE_ID='200', END_OF_PREV_DATE='2001-06-17', JOB_ID='AD_ASST'),
 Row(EMPLOYEE_ID='176', END_OF_PREV_DATE='2006-12-31', JOB_ID='SA_REP')]

In [178]:
#РЕЗУЛЬТАТ
sqlDF = spark.sql("\
  SELECT C.EMPLOYEE_ID , P.JOB_ID AS PREV_JOB, C.JOB_ID AS CUR_JOB, DATEDIFF(CAST(START_OF_CUR AS DATE),CAST(END_OF_PREV_DATE AS DATE)) AS DATE_DIFF,CAST(DATEDIFF(CAST(START_OF_CUR AS DATE),CAST(END_OF_PREV_DATE AS DATE))/7 AS INT) AS WEEK_DIFF \
  FROM PREV_JOB AS P\
  LEFT JOIN CUR AS C ON C.EMPLOYEE_ID = P.EMPLOYEE_ID\
  ")
sqlDF.createOrReplaceTempView("FINAL")
sqlDF.collect()

[Row(EMPLOYEE_ID='101', PREV_JOB='AC_ACCOUNT', CUR_JOB='AC_MGR', DATE_DIFF=1, WEEK_DIFF=0),
 Row(EMPLOYEE_ID='200', PREV_JOB='AD_ASST', CUR_JOB='AC_ACCOUNT', DATE_DIFF=379, WEEK_DIFF=54),
 Row(EMPLOYEE_ID='176', PREV_JOB='SA_REP', CUR_JOB='SA_MAN', DATE_DIFF=1, WEEK_DIFF=0)]

Выведете уникальные телефонные номера

In [ ]:
sqlDF = spark.sql("\
  SELECT DISTINCT PHONE_NUMBER   \
  FROM employees_df as e ")
sqlDF.collect()

Есть ли сотрудники с одинаковыми фамилиями и сколько их.

In [87]:
sqlDF = spark.sql("\
  SELECT * FROM (SELECT LAST_NAME, COUNT(EMPLOYEE_ID) AS CNT \
  FROM employees_df as e \
  GROUP BY LAST_NAME) WHERE CNT > 1 \
  ")
sqlDF.collect()

[Row(LAST_NAME='Smith', CNT=2),
 Row(LAST_NAME='King', CNT=2),
 Row(LAST_NAME='Cambrault', CNT=2),
 Row(LAST_NAME='Taylor', CNT=2)]